# Finetuning Ranker

# Installs

In [1]:
pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Infra

## Paths

In [6]:
PATH_TRAIN_MODEL_LOCAL = "/content/drive/MyDrive/treinamento/202301_IA368DD/indir/train/monot5"

## Função de verificação de memória

In [7]:
from psutil import virtual_memory

In [8]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jun 27 16:08:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
def mostra_memoria(lista_mem=['cpu']):
  """
  Esta função exibe informações de memória da CPU e/ou GPU, conforme parâmetros fornecidos.

  Parâmetros:
  -----------
  lista_mem : list, opcional
      Lista com strings 'cpu' e/ou 'gpu'.
      'cpu' - exibe informações de memória da CPU.
      'gpu' - exibe informações de memória da GPU (se disponível).
      O valor padrão é ['cpu'].

  Saída:
  -------
  A função não retorna nada, apenas exibe as informações na tela.

  Exemplo de uso:
  ---------------
  Para exibir informações de memória da CPU:
      mostra_memoria(['cpu'])

  Para exibir informações de memória da CPU e GPU:
      mostra_memoria(['cpu', 'gpu'])

  Autor: Marcus Vinícius Borela de Castro

  """
  if 'cpu' in lista_mem:
    vm = virtual_memory()
    ram={}
    ram['total']=round(vm.total / 1e9,2)
    ram['available']=round(virtual_memory().available / 1e9,2)
    # ram['percent']=round(virtual_memory().percent / 1e9,2)
    ram['used']=round(virtual_memory().used / 1e9,2)
    ram['free']=round(virtual_memory().free / 1e9,2)
    ram['active']=round(virtual_memory().active / 1e9,2)
    ram['inactive']=round(virtual_memory().inactive / 1e9,2)
    ram['buffers']=round(virtual_memory().buffers / 1e9,2)
    ram['cached']=round(virtual_memory().cached/1e9 ,2)
    print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")
    print('/nGPU')
    gpu_info = !nvidia-smi
  if 'gpu' in lista_mem:
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)


In [10]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 13.61
 available 12.43
 used 0.84
 free 5.54
 cached 7.16
 buffers 0.07
/nGPU
Tue Jun 27 16:08:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--

## Imports

In [11]:
import os

In [12]:
from transformers import Trainer
import numpy as np
import torch


In [13]:
import pandas as pd

In [14]:
from dataclasses import dataclass, field

In [15]:
from datasets import Dataset

In [16]:
import json
from transformers import (
    AutoTokenizer,
    AutoConfig,
    MT5Tokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
    Trainer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorWithPadding,
    DataCollatorForSeq2Seq,
)
from tqdm.auto import tqdm
from datasets import Dataset
from dataclasses import dataclass, field


## Preparando para debug e display

In [17]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [18]:
def config_debug():
  """
  Esta função configura as opções de debug do PyTorch e dos pacotes
  transformers e datasets.
  """

  # Define opções de impressão de tensores para o modo científico
  torch.set_printoptions(sci_mode=True)
  """
    Significa que valores muito grandes ou muito pequenos são mostrados em notação científica.
    Por exemplo, em vez de imprimir o número 0.0000012345 como 0.0000012345,
    ele seria impresso como 1.2345e-06. Isso é útil em situações em que os valores dos tensores
    envolvidos nas operações são muito grandes ou pequenos, e a notação científica permite
    uma melhor compreensão dos números envolvidos.
  """

  # Habilita detecção de anomalias no autograd do PyTorch
  torch.autograd.set_detect_anomaly(True)
  """
    Permite identificar operações que podem causar problemas de estabilidade numérica,
    como gradientes explodindo ou desaparecendo. Quando essa opção é ativada,
    o PyTorch verifica se há operações que geram valores NaN ou infinitos nos tensores
    envolvidos no cálculo do gradiente. Se for detectado um valor anômalo, o PyTorch
    interrompe a execução e gera uma exceção, permitindo que o erro seja corrigido
    antes que se torne um problema maior.

    É importante notar que a detecção de anomalias pode ter um impacto significativo
    no desempenho, especialmente em modelos grandes e complexos. Por esse motivo,
    ela deve ser usada com cautela e apenas para depuração.
  """

  # Configura variável de ambiente para habilitar a execução síncrona (bloqueante) das chamadas da API do CUDA.
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  """
    o Python aguarda o término da execução de uma chamada da API do CUDA antes de executar a próxima chamada.
    Isso é útil para depurar erros no código que envolve operações na GPU, pois permite que o erro seja capturado
    no momento em que ocorre, e não depois de uma sequência de operações que pode tornar a origem do erro mais difícil de determinar.
    No entanto, é importante lembrar que esse modo de execução é significativamente mais lento do que a execução assíncrona,
    que é o comportamento padrão do CUDA. Por isso, é recomendado utilizar esse comando apenas em situações de depuração
    e removê-lo após a solução do problema.
  """

  # Define o nível de verbosity do pacote transformers para info
  # transformers.utils.logging.set_verbosity_info()


  """
    Define o nível de detalhamento das mensagens de log geradas pela biblioteca Hugging Face Transformers
    para o nível info. Isso significa que a biblioteca irá imprimir mensagens de log informativas sobre
    o andamento da execução, tais como tempo de execução, tamanho de batches, etc.

    Essas informações podem ser úteis para entender o que está acontecendo durante a execução da tarefa
    e auxiliar no processo de debug. É importante notar que, em alguns casos, a quantidade de informações
    geradas pode ser muito grande, o que pode afetar o desempenho do sistema e dificultar a visualização
    das informações relevantes. Por isso, é importante ajustar o nível de detalhamento de acordo com a
    necessidade de cada tarefa.

    Caso queira reduzir a quantidade de mensagens, comentar a linha acima e
      descomentar as duas linhas abaixo, para definir o nível de verbosity como error ou warning

    transformers.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_warning()
  """


  # Define o modo verbose do xmode, que é utilizado no debug
  # %xmode Verbose

  """
    Comando usado no Jupyter Notebook para controlar o modo de exibição das informações de exceções.
    O modo verbose é um modo detalhado que exibe informações adicionais ao imprimir as exceções.
    Ele inclui as informações de pilha de chamadas completa e valores de variáveis locais e globais
    no momento da exceção. Isso pode ser útil para depurar e encontrar a causa de exceções em seu código.
    Ao usar %xmode Verbose, as informações de exceção serão impressas com mais detalhes e informações adicionais serão incluídas.

    Caso queira desabilitar o modo verbose e utilizar o modo plain,
    comentar a linha acima e descomentar a linha abaixo:
    %xmode Plain
  """

  """
    Dica:
    1.  pdb (Python Debugger)
      Quando ocorre uma exceção em uma parte do código, o programa para a execução e exibe uma mensagem de erro
      com informações sobre a exceção, como a linha do código em que ocorreu o erro e o tipo da exceção.

      Se você estiver depurando o código e quiser examinar o estado das variáveis ​​e executar outras operações
      no momento em que a exceção ocorreu, pode usar o pdb (Python Debugger). Para isso, é preciso colocar o comando %debug
      logo após ocorrer a exceção. Isso fará com que o programa pare na linha em que ocorreu a exceção e abra o pdb,
      permitindo que você explore o estado das variáveis, examine a pilha de chamadas e execute outras operações para depurar o código.


    2. ipdb
      O ipdb é um depurador interativo para o Python que oferece recursos mais avançados do que o pdb,
      incluindo a capacidade de navegar pelo código fonte enquanto depura.

      Você pode começar a depurar seu código inserindo o comando ipdb.set_trace() em qualquer lugar do
      seu código onde deseja pausar a execução e começar a depurar. Quando a execução chegar nessa linha,
      o depurador entrará em ação, permitindo que você examine o estado atual do seu programa e execute
      comandos para investigar o comportamento.

      Durante a depuração, você pode usar comandos:
        next (para executar a próxima linha de código),
        step (para entrar em uma função chamada na próxima linha de código)
        continue (para continuar a execução normalmente até o próximo ponto de interrupção).

      Ao contrário do pdb, o ipdb é um depurador interativo que permite navegar pelo código fonte em que
      está trabalhando enquanto depura, permitindo que você inspecione variáveis, defina pontos de interrupção
      adicionais e até mesmo execute expressões Python no contexto do seu programa.
  """


In [19]:
config_display()

In [20]:
config_debug()

## Vinculando pasta do google drive para salvar dados

In [21]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [22]:
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /content


## Constants

In [23]:
TOKEN_FALSE = '▁não'
TOKEN_TRUE = '▁sim'

In [24]:
MODEL_NAME = 'unicamp-dl/ptt5-base-pt-msmarco-100k-v2'
# 'unicamp-dl/mt5-3B-mmarco-en-pt'

# Carga dos dados

In [25]:
PATH_LOCAL_DATA = '/content/drive/MyDrive/treinamento/202301_IA368DD/indir/data'

In [26]:
# path_data = '/content/drive/MyDrive/treinamento/202301_IA368DD/indir/data/train_data_juris_tcu_index_bm25.csv'
# PATH_TRAIN_DATA_ZIP = f"{PATH_LOCAL_DATA}/train_data_juris_tcu_index_limit_50_per_doc.zip"
PATH_TRAIN_DATA = f"{PATH_LOCAL_DATA}/train_data_juris_tcu_index.csv"

In [27]:
os.path.exists(PATH_TRAIN_DATA)

True

In [28]:
%%time
if not os.path.exists(PATH_TRAIN_DATA):
  import zipfile
  !wget https://github.com/marcusborela/ind-ir/raw/main/data/train_juris_tcu_index/train_data_juris_tcu_index_limit_{alt_aqui}_per_doc.zip -O {PATH_TRAIN_DATA_ZIP}

  # Extrair o arquivo zip
  with zipfile.ZipFile(PATH_TRAIN_DATA_ZIP, 'r') as zip_ref:
      zip_ref.extractall(PATH_LOCAL_DATA)

  # Listar os arquivos extraídos
  arquivos_extraidos = zip_ref.namelist()
  # Exibir os arquivos extraídos
  for arquivo in arquivos_extraidos:
      print(arquivo)
  print("File loaded")
else:
  print("File already there!")

File already there!
CPU times: user 1.51 ms, sys: 0 ns, total: 1.51 ms
Wall time: 11.8 ms


In [29]:
df = pd.read_csv(PATH_TRAIN_DATA)

In [30]:
df.shape
# lim 100(111852, 6)

(402738, 7)

Verificando correção do arquivo!

In [31]:
print(df.isnull().sum())

QUERY_ID      0
DOC_ID        0
RELEVANCE     0
SCORE         0
TYPE          0
DOC_TEXT      0
QUERY_TEXT    0
Length: 7, dtype: int64


In [32]:
df[['QUERY_TEXT','DOC_TEXT']].applymap(len).describe()

,QUERY_TEXT,DOC_TEXT
count,402738.0000000,402738.0000000
mean,322.8252313,830.6957451
std,165.8299958,398.1844365
min,41.0000000,86.0000000
25%,217.0000000,572.0000000
50%,294.0000000,759.0000000
75%,391.0000000,1020.0000000
max,4212.0000000,3739.0000000


Para cada positivo, tem 5 negativos

In [33]:
df['RELEVANCE'].describe()

count    402738.0000000
mean          0.1666667
std           0.3726785
min           0.0000000
25%           0.0000000
50%           0.0000000
75%           0.0000000
max           1.0000000
Name: RELEVANCE, Length: 8, dtype: float64

In [34]:
df.head()

,QUERY_ID,DOC_ID,RELEVANCE,SCORE,TYPE,DOC_TEXT,QUERY_TEXT
0,151655,1943,1,0.897,TEMA,"O termo é ""Agente público"".\nAgente público te...",O dever de observância à hierarquia militar nã...
1,151655,15441,0,0.732,"relevant:TEMA, not relevant:TOTAL","O termo é ""Reforma-prêmio"".\nReforma-prêmio te...",O dever de observância à hierarquia militar nã...
2,151655,6373,0,0.717,"relevant:TEMA, not relevant:TOTAL","O termo é ""Exercício financeiro anterior"".\nEx...",O dever de observância à hierarquia militar nã...
3,151655,6973,0,0.680,"relevant:TEMA, not relevant:TOTAL","O termo é ""CJF"".\nCJF é classificado como uma ...",O dever de observância à hierarquia militar nã...
4,151655,7201,0,0.751,"relevant:TEMA, not relevant:TOTAL","O termo é ""Embratur"".\nEmbratur é classificado...",O dever de observância à hierarquia militar nã...


In [35]:
df["label"] = [TOKEN_FALSE if relevance == 0 else TOKEN_TRUE for relevance in df["RELEVANCE"]]

In [36]:
df.head()

,QUERY_ID,DOC_ID,RELEVANCE,SCORE,TYPE,DOC_TEXT,QUERY_TEXT,label
0,151655,1943,1,0.897,TEMA,"O termo é ""Agente público"".\nAgente público te...",O dever de observância à hierarquia militar nã...,▁sim
1,151655,15441,0,0.732,"relevant:TEMA, not relevant:TOTAL","O termo é ""Reforma-prêmio"".\nReforma-prêmio te...",O dever de observância à hierarquia militar nã...,▁não
2,151655,6373,0,0.717,"relevant:TEMA, not relevant:TOTAL","O termo é ""Exercício financeiro anterior"".\nEx...",O dever de observância à hierarquia militar nã...,▁não
3,151655,6973,0,0.680,"relevant:TEMA, not relevant:TOTAL","O termo é ""CJF"".\nCJF é classificado como uma ...",O dever de observância à hierarquia militar nã...,▁não
4,151655,7201,0,0.751,"relevant:TEMA, not relevant:TOTAL","O termo é ""Embratur"".\nEmbratur é classificado...",O dever de observância à hierarquia militar nã...,▁não


In [37]:
df.rename(columns={'DOC_TEXT': 'text', 'QUERY_TEXT':'query'},inplace=True)

In [38]:
df.head()

,QUERY_ID,DOC_ID,RELEVANCE,SCORE,TYPE,text,query,label
0,151655,1943,1,0.897,TEMA,"O termo é ""Agente público"".\nAgente público te...",O dever de observância à hierarquia militar nã...,▁sim
1,151655,15441,0,0.732,"relevant:TEMA, not relevant:TOTAL","O termo é ""Reforma-prêmio"".\nReforma-prêmio te...",O dever de observância à hierarquia militar nã...,▁não
2,151655,6373,0,0.717,"relevant:TEMA, not relevant:TOTAL","O termo é ""Exercício financeiro anterior"".\nEx...",O dever de observância à hierarquia militar nã...,▁não
3,151655,6973,0,0.680,"relevant:TEMA, not relevant:TOTAL","O termo é ""CJF"".\nCJF é classificado como uma ...",O dever de observância à hierarquia militar nã...,▁não
4,151655,7201,0,0.751,"relevant:TEMA, not relevant:TOTAL","O termo é ""Embratur"".\nEmbratur é classificado...",O dever de observância à hierarquia militar nã...,▁não


In [39]:
df = df[['query', 'text', 'label']]


In [40]:

df.shape

(402738, 3)

In [41]:
import gc

# ... código anterior ...

# Liberar memória utilizando gc.collect()
gc.collect()

125

# Separating evaluation data and prepare dataset tokenized

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
train_df, valid_df = train_test_split(df, test_size=0.005,
                                      stratify=df['label'].values, random_state=123)
# Definir os argumentos de treinamento

In [44]:
train_df.shape, valid_df.shape

((400724, 3), (2014, 3))

In [45]:
valid_df[:2]

,query,text,label
391682,O início de obra pública sem a contratação de ...,"O termo é ""TCU"".\nTCU é classificado como uma ...",▁não
153097,Os efeitos de decisão judicial em ação ordinár...,"O termo é ""CCHA"".\nCCHA é classificado como um...",▁não


In [46]:
print(np.unique(train_df['label'], return_counts=True), '\n', np.unique(valid_df['label'], return_counts=True))

(array(['▁não', '▁sim'], dtype=object), array([333937,  66787])) 
 (array(['▁não', '▁sim'], dtype=object), array([1678,  336]))


In [47]:
train_dataset = Dataset.from_pandas(train_df[["query", "text", "label"]].reset_index(drop=True))
valid_dataset = Dataset.from_pandas(valid_df[["query", "text", "label"]].reset_index(drop=True))

In [48]:
len(train_dataset), len(valid_dataset)

(400724, 2014)

In [49]:
valid_dataset[0]

{'query': 'O início de obra pública sem a contratação de empresa supervisora para subsidiar o acompanhamento e a fiscalização da execução contratual, nos casos em que a complexidade e a importância do empreendimento o exijam, afronta o art. 67 da Lei 8.666/1993 e enseja a responsabilização do gestor omisso por eventuais irregularidades verificadas no contrato.',
 'text': 'O termo é "TCU".\nTCU é classificado como uma organização.\nTCU tem nota de escopo: "Em 7 de novembro de 1890, por iniciativa do então Ministro da Fazenda, Rui Barbosa, o Decreto nº 966-A criou o Tribunal de Contas da União, norteado pelos princípios da autonomia, fiscalização, julgamento, vigilância e energia.".\nTCU tem sinônimo: "Tribunal de Contas da União", "EFS Tribunal de Contas da União", "EFS Brasil", "Tribunal de Contas do Brasil", "Tribunal da União", "Tribunal de Cuentas de Brasil", "Federal Court of Accounts", "Tribunal de Cuentas de la Unión", "Tribunal de Cuentas de la Union Brasil", "Tribunal de Contas

In [50]:
del df, train_df, valid_df


In [51]:
gc.collect()

20

# Tokenize

In [52]:
# tokenizer = MT5Tokenizer.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [53]:
def tokenize(batch):
    queries_documents = [f"Query: {query} Document: {text} Relevant:" for query, text in zip(batch["query"], batch["text"])]
    print(f"Chamado tokenize len(queries_documents): {len(queries_documents)}")
    tokenized = tokenizer(
        queries_documents,
        padding=True, # "max_length",
        truncation=True,
        # return_tensors="pt",
        max_length= 512
    )
    # tokenized["labels"] = [[label] for label in batch["label"]]
    # tokenized['label'] = [[token_false, token_true][int(pairs["label"][i])]
    tokenized["labels"] = tokenizer(batch['label'])['input_ids']
    # tokenized["labels"] = [tokenizer.get_vocab()[token] for token in batch['label']]
    # tokenized["labels"] = [token_id_true if label == 'true' else token_id_false for label in batch["label"]]
    return tokenized


In [54]:
%%time
# valid_dataset.set_transform(tokenize)
valid_dataset = valid_dataset.map(
        tokenize,
        remove_columns=('query', 'text', 'label'),
        batched=True,
        desc='Tokenizing',
    )

Tokenizing:   0%|          | 0/2014 [00:00<?, ? examples/s]

Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 14
CPU times: user 2.75 s, sys: 79.5 ms, total: 2.83 s
Wall time: 1.85 s


In [55]:
%%time
# train_dataset.set_transform(tokenize)
train_dataset = train_dataset.map(
        tokenize,
        remove_columns=('query', 'text', 'label'),
        batched=True,
        desc='Tokenizing',
    )

Tokenizing:   0%|          | 0/400724 [00:00<?, ? examples/s]

Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_docum

In [56]:
print(valid_dataset[0])

{'input_ids': [2094, 540, 46, 28, 291, 4, 516, 1450, 203, 7, 10483, 4, 449, 19475, 44, 20, 575, 6, 6334, 256, 9, 11911, 8, 7, 18522, 11, 3221, 132, 6935, 3, 84, 1261, 12, 13, 7, 9213, 8, 7, 1567, 10, 9738, 9, 239, 11544, 41, 3, 6833, 1459, 9, 7904, 5, 8966, 11, 1090, 290, 5, 975, 2994, 113, 21916, 8, 833, 32, 570, 7, 21581, 93, 10, 18666, 9, 12159, 43, 26, 20868, 20156, 7972, 178, 19, 1170, 5, 745, 797, 1806, 46, 28, 762, 21, 15, 12422, 746, 64, 299, 165, 746, 21, 3823, 29, 17, 1001, 5, 299, 165, 746, 87, 2649, 4, 19414, 46, 15, 4169, 280, 4, 362, 4, 5963, 3, 26, 3985, 10, 174, 3596, 11, 5679, 3, 6991, 5640, 3, 9, 4635, 19, 351, 2994, 14, 89, 1682, 9, 2330, 4, 3908, 6, 11, 842, 3, 220, 338, 210, 3862, 11, 6700, 3, 18522, 3, 4424, 3, 14263, 8, 1033, 5, 64, 299, 165, 746, 87, 13059, 46, 15, 22502, 77, 4, 3908, 6, 11, 842, 49, 15, 15962, 207, 2330, 4, 3908, 6, 11, 842, 49, 15, 15962, 207, 112, 49, 15, 22502, 77, 4, 3908, 6, 10, 112, 49, 15, 22502, 77, 11, 842, 49, 15, 22502, 77, 4, 1397, 

# Train

## setup

In [57]:
PATH_TRAIN_MODEL_LOCAL

'/content/drive/MyDrive/treinamento/202301_IA368DD/indir/train/monot5'

In [58]:
num_step_alert = 200
training_args = Seq2SeqTrainingArguments(output_dir=PATH_TRAIN_MODEL_LOCAL)
# Needed to make the Trainer work with an on-the-fly transformation on the dataset
# training_args.remove_unused_columns = False
training_args.output_dir = PATH_TRAIN_MODEL_LOCAL
training_args.warmup_steps=200 # Alterar!
training_args.num_train_epochs=1.0 # Alterar!
training_args.logging_steps=num_step_alert # Alterar!
training_args.save_strategy="steps"
training_args.save_steps=num_step_alert
training_args.save_total_limit=10
training_args.learning_rate=5e-5
training_args.per_device_train_batch_size=8 # t4: 8, a100-40: 32
training_args.gradient_accumulation_steps=4 # t4: 4, a100-40: 2
#training_args._n_gpu = 1
# training_args.bf16 = True # se for usar a100, 3090, 4090 -> usar
training_args.ignore_data_skip = True
training_args.load_best_model_at_end = True
training_args.evaluation_strategy='steps'
training_args.eval_steps=num_step_alert
training_args.do_eval = True
# training_args.optim='adamw_hf' #default
training_args.gradient_checkpointing = False # True
# se precisar economizar gpu
# training_args.optim='adamw_bnb_8bit'
# training_args.gradient_checkpointing = True
# training_args.report_to="none"

In [59]:
!ls

drive  sample_data


In [60]:
print(training_args)

Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=200,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
generation_config=None,
generation_max_length=None,
generation_num_beams=None,
gradient_accumulation_steps=4,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backe

In [61]:
# from transformers.integrations import NeptuneCallback
# rastro_neptune = NeptuneRastroRun(hparam, parm_lista_tag= tag_contexto_rastro)
# neptune_callback = NeptuneCallback(run=rastro_neptune)

In [62]:
%%time
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)


CPU times: user 2.68 s, sys: 1.34 s, total: 4.02 s
Wall time: 7.99 s


In [63]:
trainer_cls = Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [64]:
# Limpa o cache da memória da GPU
# del trainer
torch.cuda.empty_cache()

In [65]:
gc.collect()

73

## Train

In [66]:
raise Exception('Parar aqui reinício!')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
Exception: Parar aqui reinício!

In [67]:
trainer = trainer_cls(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
train_metrics = trainer.train(resume_from_checkpoint=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


In [ ]:
train_metrics = trainer.train(resume_from_checkpoint=True)

In [ ]:
train_metrics = trainer.train(resume_from_checkpoint=False)

In [ ]:
EXECUÇÕES ANTERIORES

In [ ]:
%%time
train_metrics = trainer.train(resume_from_checkpoint=True)

In [ ]:
print(train_metrics )

Não sei se a mudança do batch size (32x2) 64 para 32 (8x4) impactou passar por dados duas vezes!!!

In [ ]:
%%time
train_metrics = trainer.train(resume_from_checkpoint=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
train_metrics = trainer.train(resume_from_checkpoint=True)

Abaixo execucao lim50: a100/40gb (erro mount drive)

In [ ]:
%%time
train_metrics = trainer.train(resume_from_checkpoint=False)

In [ ]:
train_metrics = trainer.train(resume_from_checkpoint=True)

In [ ]:
%%time
train_metrics = trainer.train(resume_from_checkpoint=False)

In [ ]:
train_metrics = trainer.train(resume_from_checkpoint=True)

In [ ]:
train_metrics

In [ ]:
huggingface-cli login

In [ ]:
pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()


In [ ]:
trainer.push_to_hub(
    model_id="ptt5-base-pt-msmarco-100k-v2-indir-lim100",
    repo_name="marcusborela"
    # use_auth_token=""
)